In [1]:
from Tesina_Sentinel_Utils import S2Band
 
start_date = '2019-01-01'
end_date = '2019-12-31'
bands = selected_bands = ['blue', 'green', 'red', 'nir', 'nir08', 'nir09', 'rededge1', 'rededge2', 'rededge3', 'swir16', 'swir22']


input_path = './data/input/test/'
cache_path = './data/input/cache/'
models_path = './data/output/models/'

stac_cfg = {
    'sentinel-2-l2a': {
        'assets': {'*': {'data_type': 'uint16', 'nodata': 0}},
    }
}


In [2]:

class ZoneData:
    def __init__(self):
        self.zones = {}
        self.zone_names = []

    def add_zone(self, name):
        if name not in self.zones:
            self.zones[name] = {
                "names": [],
                "polygons": []
            }
            self.zone_names.append(name)

    def add_polygon(self, zone_name, polygon_name, polygon):
        if zone_name in self.zones:
            self.zones[zone_name]["names"].append(polygon_name)
            self.zones[zone_name]["polygons"].append(polygon)

    def get_zone_by_index(self, index):
        try:
            zone_name = self.zone_names[index]
            return self.zones[zone_name]
        except IndexError:
            return None
        
    def get_zone_by_name(self, name):
        if name in self.zones:
            return self.zones[name]
        return None
    
    def get_zone_name_by_index(self, index):
        try:
            return self.zone_names[index]
        except IndexError:
            return None

    def get_zones_list(self):
        return self.zone_names

    def get_polygon_list(self):
        polygons = []
        for zone_name in self.zones:
            polygons += self.zones[zone_name]["names"]
        return polygons
        
    def get_polygon_names_by_zone_name(self, zone_name):
        if zone_name in self.zones:
            return self.zones[zone_name]["names"]
        return None

    def get_polygon_names_by_zone_index(self, index):
        try:
            zone_name = self.zone_names[index]
            return self.zones[zone_name]["names"]
        except IndexError:
            return None

    def get_polygon_by_index(self, index, zone_name = None):
        if zone_name is None:
            i = 0
            for zone_name in self.zones:
                for polygon in self.zones[zone_name]["polygons"]:
                    if i == index:
                        return polygon
                    i += 1
        else:
            if zone_name in self.zones and index < len(self.zones[zone_name]["polygons"]):
                return self.zones[zone_name]["polygons"][index]
        return None
    
    def get_polygon_by_name(self, name):
        for zone_name in self.zones:
            for polygon_name, polygon in zip(self.zones[zone_name]["names"], self.zones[zone_name]["polygons"]):
                if polygon_name == name:
                    return polygon
        return None
        



In [3]:

from shapely.geometry import shape, Polygon
from shapely.wkt import loads
from lxml import etree
from fastkml import  kml
import os

def process_kml_files(folder_path):
    zone_data = ZoneData()
    
    # List all .kml files in the given directory
    files = [f for f in os.listdir(folder_path) if f.endswith('.kml')]
    
    for file in files:
        print(f'Processing file: {file}')
        filepath = os.path.join(folder_path, file)
        
        # Attempt to open and parse the KML file
        try:
            with open(filepath, "r", encoding='utf-8', errors='ignore') as file_content:
                doc = file_content.read()
            k = kml.KML()
            k.from_string(doc.encode('utf-8'))
        except etree.ParseError as e:
            print(f"Unable to parse file: {file}. Error: {e}")
            continue
        
        # Assuming each KML file has a structure where the first feature is a Folder
        # and the Folder contains a set of Placemarks
        main_folder = list(list(k.features())[0].features())[0]  # Access the first Folder in the KML
        zone_name = main_folder.name
        print(f'Processing folder: {main_folder.name}')                    
        zone_data.add_zone(zone_name)

        for placemark in main_folder.features():
            print(f'Processing placemark: {placemark.name}')
            # If the placemark has a polygon, process it
            if placemark.geometry:
                if isinstance(placemark.geometry, Polygon):
                    # Add zone and polygon to the ZoneData object
                    zone_data.add_polygon(zone_name, placemark.name, placemark.geometry)
                    print("Bounding box: {}".format(placemark.geometry.bounds)) 
                else:
                    shapely_polygon = shape(placemark.geometry.__geo_interface__)
                    zone_data.add_polygon(zone_name, placemark.name, shapely_polygon)

    return zone_data  # Ensure to return the ZoneData object after processing all files


In [4]:
zone_data = process_kml_files("./zones/")
zone_index = 1
zone_by_index = zone_data.get_zone_name_by_index(zone_index)
print(f"Zona en índice {zone_index}:", zone_by_index)
polygon = zone_data.get_polygon_by_index(0, zone_by_index)
print("Polígono:", polygon)

Processing file: Bosques Bio Bio.kml
Processing folder: Bosques Bio Bio
Processing placemark: Bosque 1
Processing placemark: Bosque 2
Processing placemark: Bosque 3
Processing file: Incendios.kml
Processing folder: Incendios
Processing placemark: Chiguayante 1
Processing placemark: Chiguayante 2
Processing placemark: Chiguayante 3
Processing placemark: Chiguayante 4
Processing placemark: Santa Juana Oeste 1A
Processing placemark: Santa Juana Oeste 2A
Processing placemark: Santa Juana Oeste 3A
Processing placemark: Santa Juana Oeste 4A
Processing placemark: Santa Juana Oeste 5A
Processing placemark: Santa Juana Oeste 6A
Processing placemark: Santa Juana Oeste 7A
Processing file: Bosques Arauco.kml
Processing folder: Bosques Arauco
Processing placemark: Bosque 1
Processing placemark: Bosque 2
Processing file: Provoque.kml
Processing folder: Provoque
Processing placemark: Tala 1
Processing placemark: Tala 2
Processing placemark: Tala 3
Processing placemark: Tala 4
Processing placemark: Ta

In [5]:
from pystac_client import Client
from odc.stac import configure_rio, stac_load
import dask.distributed

catalog = Client.open("https://earth-search.aws.element84.com/v1")
client = dask.distributed.Client()
configure_rio(cloud_defaults=True, aws={"aws_unsigned": True}, client=client)
display(client)

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 5
Total threads: 20,Total memory: 15.50 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37943,Workers: 5
Dashboard: http://127.0.0.1:8787/status,Total threads: 20
Started: Just now,Total memory: 15.50 GiB
Comm: tcp://127.0.0.1:46185,Total threads: 4
Dashboard: http://127.0.0.1:37633/status,Memory: 3.10 GiB
Nanny: tcp://127.0.0.1:33049,


In [6]:
import os
from Tesina_General_Utils import Logger, LogLevel, make_dir
from datetime import datetime, timedelta
import hashlib
import numpy as np
import xarray as xr
from PIL import Image
from Tesina_Images_Utils import normalize_image_percentile, create_mask_from_polygon

def fetch_or_cache_stac_data_by_date(catalog, bands, collections, date:str, bbox, resolution, stac_config=None, crs="EPSG:4326", cache_dir='data_cache', log_level: LogLevel = LogLevel.Trace, save_rgb_image=False):
        
    dims = None

    logger = Logger(min_log_level=log_level)
    target_date = datetime.strptime(date, "%Y-%m-%d")
    delta_days = 15
    
    # Get the start and end dates
    start_date = target_date - timedelta(days=delta_days)
    end_date = target_date + timedelta(days=delta_days)


    metadata_str = f"{collections}_{bands}_{date}_{bbox}_{stac_config}_{resolution}_{crs}"
    metadata_hash = hashlib.md5(metadata_str.encode()).hexdigest()
    logger.trace(f"Metadata hash: {metadata_hash}")
    cache_filepath = os.path.join(cache_dir, f"{metadata_hash}.nc")

    if os.path.exists(cache_filepath):
        bands_data  = xr.open_dataset(cache_filepath)
        logger.info(f"Data for bands {bands} loaded from cache")
    else:
        make_dir([cache_dir])
        query = catalog.search(
            collections=collections,
            datetime=f"{start_date.strftime('%Y-%m-%d')}/{end_date.strftime('%Y-%m-%d')}",
            bbox=bbox,
            query={'eo:cloud_cover': {'lt': 100}} if collections == ["sentinel-2-l2a"] else {}
        )

        logger.info(f"Fetching data for bands {bands} from STAC")
        items = list(query.items())
        data = stac_load(items, resolution=10, bands=bands, bbox=bbox, stac_cfg=stac_config, groupby='solar_day', crs=crs)
        date_data = data.sel(time=target_date, method='nearest')

            
        #print shape (w,h)
        if dims is None:
            dims = date_data.dims
        

        # Compute and save each band                    
        logger.info(f"Saving data for bands {bands}")   
        bands_data = date_data.compute()

        if 'time' not in bands_data.coords:
            logger.error(f"Data for bands does not have a 'time' dimension. Here are the coordinates: {bands_data.coords}")
            return None             

        bands_data.to_netcdf(cache_filepath, engine='netcdf4')
    
        
    rgb_image_path = None
    if save_rgb_image and len(bands_data) > 0:
        # Check if the RGB image is in group_data_list
        rgb_bands = ['red', 'green', 'blue']
        if all(band in bands_data.data_vars for band in rgb_bands):
            # Combine RGB bands into a single DataArray
            rgb_arrays = [bands_data[band] for band in rgb_bands]
            rgb_data = xr.concat(rgb_arrays, dim='band')

            # Now rgb_data should have a 'band' dimension to transpose
            rgb_data = rgb_data.transpose('y', 'x', 'band')
            rgb_data = normalize_image_percentile(rgb_data)

            # Normalize and convert data for image saving
            rgb_data = rgb_data.astype(np.float32)
            rgb_data /= rgb_data.max()  # Normalize to the maximum reflectance
            rgb_data = rgb_data.clip(0, 1)
            rgb_image_data = (rgb_data * 255).astype(np.uint8).values

            # Create and save the RGB image
            rgb_image = Image.fromarray(rgb_image_data, 'RGB')
            rgb_image_path = f"{cache_filepath.replace('.nc', '')}_rgb.png"
            rgb_image.save(rgb_image_path)
            
            logger.info(f"RGB image saved to {rgb_image_path}")
            
    return bands_data, rgb_image_path

In [7]:
from shapely.affinity import affine_transform
import numpy as np
from skimage.measure import find_contours
from shapely.geometry import Polygon, mapping
import geopandas as gpd

def get_transformation_matrix(polygon_bound, mask_shape):
    """
    Calculate the transformation matrix based on the polygon_bound and mask_shape.
    """
    minx, miny, maxx, maxy = polygon_bound.bounds  # Obtaining the bounds of the polygon
    y_pixels, x_pixels = mask_shape

    x_scale = (maxx - minx) / x_pixels
    y_scale = (maxy - miny) / y_pixels

    # Crear una matriz de transformación para convertir las coordenadas de píxeles a coordenadas geográficas
    # [x_scale, 0, 0, y_scale, minx, miny]
    transformation_matrix = [x_scale, 0, 0, -y_scale, minx, maxy]  

    return transformation_matrix


def get_polygons_from_mask(mask, label, polygon_bound, zone, region, type_index, date, ref_img_date = None, ref_filename = None, ref_prev_img_filename = None, print_info = False):
    """
    Convert a mask to a list of polygons with metadata.

    Parameters:
    mask (numpy array or xarray): A binary mask to convert to polygons.
    polygon_bound (Polygon): The bounding polygon.

    Returns:
    list: A list of transformed polygons.
    """
    polygons = []
    polygons_in_mask = []

    # Check if mask is an xarray, and if so, extract the numpy array
    if hasattr(mask, 'values'):
        mask_values = mask.values
    else:
        mask_values = mask

    # Making the first and last row and column zero
    mask_values[0, :] = 0
    mask_values[-1, :] = 0
    mask_values[:, 0] = 0
    mask_values[:, -1] = 0

    mask_values = np.transpose(mask_values)

    contours = find_contours(mask_values, level=0.5)
    transformation_matrix = get_transformation_matrix(polygon_bound, mask.shape)  # Define this function

    for contour in contours:
        # Convert contour coordinates to polygon
        poly = Polygon(contour)

        # Apply the transformation
        poly_transformed = affine_transform(poly, transformation_matrix)
        polygons.append({
            'geometry': poly_transformed,
        })
        polygons_in_mask.append(
            {
                'geometry': poly,
                'label': label,
            }
        )

    polygons = associate_holes(polygons)  # Function to classify polygons as 'outer' or 'hole'
    polygons_in_mask = associate_holes(polygons_in_mask)
    
    if print_info:
        for poly_data  in polygons_in_mask:
            poly = poly_data['geometry']

            # Verificar si el polígono está cerrado (en Shapely, todos los Polygon son cerrados por defecto)
            is_closed = poly.is_closed  # Esto siempre será True para objetos Polygon

            # Calcular el área del polígono
            area = poly.area

            # Imprimir la información
            print(f"Polygon: {poly_data}")
            print(f"   Is closed: {is_closed}")
            print(f"   Area: {area}")
            coords = list(poly.exterior.coords)
            if len(polygons_in_mask) == 1:
                print("Polygon coordinates:")
                for coord in coords:
                    print(coord)
                print("Is the polygon closed (first and last point the same)?", coords[0] == coords[-1])

    for poly_data in polygons:
        if zone is not None:
            poly_data.update({
                'zone': zone
            })
            
        if region is not None:
            poly_data.update({
                'region': region
            })
            
        if type_index is not None:
            poly_data.update({
                'type_index': type_index
            })
            
        if date is not None:
            poly_data.update({
                'date': date
            })
            
        if label is not None:
            poly_data.update({
                'label': label
            })
            
        if ref_filename is not None:
            poly_data.update({
                'tif': ref_filename
            })
        
        if ref_img_date is not None:
            poly_data.update({
                'prev_date': ref_img_date
            })
            
        if ref_prev_img_filename is not None:
            poly_data.update({
                'prev_tif': ref_prev_img_filename
            })

    return polygons, polygons_in_mask

def associate_holes(polygons):
    """
    Associate holes with their corresponding outer polygons.
    """
    associated_polygons_data = []
    
    for poly_data in polygons:
        poly = poly_data['geometry']  # Accessing the polygon

        # Identifying outer polygons
        if not any(poly.within(p['geometry']) for p in polygons if p != poly_data):
            # If it's an outer polygon, check for holes
            holes = [p['geometry'] for p in polygons if p['geometry'].within(poly) and p != poly_data]
            
            # Create a new polygon with the identified holes
            if holes:
                holes_coords = [hole.exterior.coords[:] for hole in holes]
                new_poly = Polygon(poly.exterior.coords[:], holes=holes_coords)
            else:
                new_poly = poly
            
            # Keep the metadata and update the geometry
            new_poly_data = poly_data.copy()  # Copy the metadata
            new_poly_data['geometry'] = new_poly  # Update the geometry
            associated_polygons_data.append(new_poly_data)
                
    return associated_polygons_data


In [8]:
import numpy as np

def normalize_global_pixels(output_data):
    """
    Normalize pixel values globally across all entries in output_data.
    
    Parameters:
    - output_data : list of dicts
        Each entry contains 'pixels' which is a list of lists of pixel values to be normalized.
    
    Returns:
    - output_data : list of dicts
        The input list of dictionaries, but with an added key 'pixels_normalized' in each
        dictionary containing the normalized pixel values.
    """
    
    # Step 1: Find global minimum and maximum pixel values for each band
    all_pixels = [pixel for entry in output_data for pixel in entry['pixels']]
    all_pixels = np.array(all_pixels)
    
    min_global = np.min(all_pixels, axis=(0, 1))
    max_global = np.max(all_pixels, axis=(0, 1))
    
    # Step 2: Normalize all pixel values using global min and max for each band
    # Step 3: Add normalized pixel values back to metadata
    for entry in output_data:
        pixels = np.array(entry['pixels'])
        pixels_normalized = (pixels - min_global) / (max_global - min_global)
        entry['pixels_normalized'] = pixels_normalized.tolist()
        
    return output_data

In [9]:
def preprocess_image_data(dataset, selected_bands, min_vals=None, max_vals=None):
    """
    Preprocess new image data from an xarray.Dataset for RandomForestClassifier.

    :param dataset: xarray.Dataset with image data.
    :param selected_bands: List of selected bands used in the model.
    :param min_vals: NumPy array or None. If None, min values will be calculated.
    :param max_vals: NumPy array or None. If None, max values will be calculated.
    :return: 2D NumPy array suitable for RandomForestClassifier.
    """
    all_pixels = []
    print(selected_bands)
    if min_vals is None or max_vals is None:
        print("Calculating min and max values for each band")
        min_vals = []
        max_vals = []
        for band in selected_bands:
            band_data = dataset[band].values
            min_vals.append(np.min(band_data))
            max_vals.append(np.max(band_data))
            print(f"Band {band}: min = {min_vals[-1]}, max = {max_vals[-1]}")

    for i, band in enumerate(selected_bands):
        band_data = dataset[band].values.ravel()
        band_normalized = (band_data - min_vals[i]) / (max_vals[i] - min_vals[i])
        all_pixels.append(band_normalized)

    return np.column_stack(all_pixels)

# def preprocess_image_data(dataset, selected_bands, min_vals=None, max_vals=None):
#     """
#     Preprocess new image data from an xarray.Dataset for RandomForestClassifier.

#     :param dataset: xarray.Dataset with image data.
#     :param selected_bands: List of selected bands used in the model.
#     :param min_vals: NumPy array or None. If None, min values will be calculated.
#     :param max_vals: NumPy array or None. If None, max values will be calculated.
#     :return: 2D NumPy array suitable for RandomForestClassifier.
#     """
#     all_pixels = []
#     print(selected_bands)
#     dataset_band_order = list(dataset.keys())

#     print(len(dataset_band_order))
#     print(len(selected_bands))
#     print(dataset_band_order)
#     print(len(min_vals))
#     print(len(max_vals))

#     # Create mappings for min and max values based on dataset's band order
#     if min_vals is None or max_vals is None:
#         print("Calculating min and max values for each band")
#         min_vals = {band: np.min(dataset[band].values) for band in dataset_band_order}
#         max_vals = {band: np.max(dataset[band].values) for band in dataset_band_order}
#         for band in selected_bands:
#             print(f"Band {band}: min = {min_vals[band]}, max = {max_vals[band]}")
#     else:
#         min_vals = dict(zip(dataset_band_order, min_vals))
#         max_vals = dict(zip(dataset_band_order, max_vals))

#     for band in selected_bands:
#         band_data = dataset[band].values.ravel()
#         band_normalized = (band_data - min_vals[band]) / (max_vals[band] - min_vals[band])
#         all_pixels.append(band_normalized)

#     return np.column_stack(all_pixels)


In [10]:
def count_pixels_by_label(model_output, label_classes):
    """
    Count the number of pixels for each label in the model output.

    :param model_output: Array of labels predicted by the model.
    :param label_classes: List of label classes.
    :return: Dictionary with counts for each label.
    """
    counts = {label: 0 for label in label_classes}
    for label in model_output:
        label_name = label_classes[label]
        counts[label_name] += 1
    return counts

def count_label_changes(output_start, output_end, label_classes):
    """
    Count how many pixels changed labels between two images and provide detailed breakdown.

    :param output_start: Array of labels from the first image.
    :param output_end: Array of labels from the second image.
    :param label_classes: List of label classes.
    :return: Dictionary with counts of label changes for each label class.
    """
    changes = {}
    for label_start, label_end in zip(output_start, output_end):
        if label_start != label_end:
            change = f"{label_classes[label_start]}->{label_classes[label_end]}"
            if change not in changes:
                changes[change] = 0
            changes[change] += 1
    return changes

In [11]:

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def show_rgb_images(output, image_path_start, image_path_end, label_classes=None, polygons_start=None, polygons_end=None, pixels_by_label_start=None, pixels_by_label_end=None):
    with output:
        output.clear_output(wait=True)
        
        nrows = 1            

        # Si hay polígonos para mostrar, configurar dos filas de ejes
        if polygons_start or polygons_end:
            n_rows = 2
            fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(15, 10), squeeze=False)
            axes_to_use = axes[1]  # Usar la segunda fila para dibujar polígonos
        else:
            # Solo una fila de ejes si no hay polígonos
            fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 6), squeeze=False)
            axes_to_use = axes[0]  # Solo hay una fila, usar esa
        
        # Cargar las imágenes
        img_start = mpimg.imread(image_path_start)
        img_end = mpimg.imread(image_path_end)

        # Mostrar imágenes originales
        axes[0, 0].imshow(img_start)
        axes[0, 0].set_title("Imagen Inicial")
        axes[0, 0].axis('off')

        axes[0, 1].imshow(img_end)
        axes[0, 1].set_title("Imagen Final")
        axes[0, 1].axis('off')
        
        # Dibujar polígonos si están presentes
        if polygons_start or polygons_end:
            axes_to_use[0].imshow(img_start)
            axes_to_use[0].set_title("Imagen Inicial con Máscaras")
            axes_to_use[0].axis('off')

            axes_to_use[1].imshow(img_end)
            axes_to_use[1].set_title("Imagen Final con Máscaras")
            axes_to_use[1].axis('off')

            if polygons_start:
                draw_polygons_on_axes(axes[1, 0], polygons_start, label_classes, img_start.shape[:2], pixels_by_label_start)
            if polygons_end:
                draw_polygons_on_axes(axes[1, 1], polygons_end, label_classes, img_end.shape[:2], pixels_by_label_end)

        fig.tight_layout()
        plt.show()



from matplotlib.patches import Polygon as mplPolygon
from matplotlib.patches import PathPatch
from matplotlib.path import Path

def calculate_net_area(poly):
    # Asegurarse de que el polígono es válido
    if not poly.is_valid:
        print("Polígono inválido:", poly)
        return 0

    exterior_area = poly.exterior.area
    holes_area = sum([hole.area for hole in poly.interiors])
    net_area = exterior_area - holes_area
    return max(net_area, 0)  # Evitar áreas negativas

import numpy as np

def draw_polygons_on_axes(axes,  polygons, label_classes, image_shape, pixel_counts=None, no_class_label='No Clasificado', no_class_color='black'):
    colors = ['yellow', 'purple', 'pink', 'cyan', 'lime']
    legend_handles = []
    added_labels = set()
    
    # Convertir label_classes a una lista si es un array de NumPy
    if isinstance(label_classes, np.ndarray):
        label_classes_list = label_classes.tolist()
    else:
        label_classes_list = label_classes

    label_classes_list = label_classes.tolist() if isinstance(label_classes, np.ndarray) else label_classes if label_classes is not None else []

    # Obtener el índice para "No Clasificado" si existe en label_classes
    no_class_index = label_classes_list.index(no_class_label) if no_class_label in label_classes_list else -1
    
    if polygons is not None:
        for poly_data in polygons:
            label_index = poly_data['label']
            # Verifica si el polígono corresponde a "No Clasificado"
            if label_index == no_class_index:
                color = no_class_color
                alpha = 1.0  # Opacidad al 100%
            else:
                color = colors[label_index % len(colors)]
                alpha = 0.25  # Transparencia aplicada a los demás colores
            
            poly = poly_data['geometry']
            
            # Solo procesar polígonos dentro del rango de la imagen
            exterior_coords = [(x, y) for x, y in poly.exterior.coords if 0 <= x < image_shape[1] and 0 <= y < image_shape[0]]
            if not exterior_coords: 
                continue

            line, = axes.plot(*zip(*exterior_coords), color=color, linewidth=1, alpha=0.8)
            axes.fill(*zip(*exterior_coords), color=color, alpha=alpha)

            for interior in poly.interiors:
                interior_coords = [(x, y) for x, y in interior.coords if 0 <= x < image_shape[1] and 0 <= y < image_shape[0]]
                if interior_coords:
                    axes.plot(*zip(*interior_coords), color=color, linewidth=1, alpha=0.8)

            # Añadir a la leyenda solo si el polígono es dibujado
            if label_classes_list is not None and label_index not in added_labels:
                legend_handles.append(line)
                added_labels.add(label_index)
    else:
        label_classes_list = None
        
    
    if pixel_counts is not None:
        # Construir leyendas solo para etiquetas añadidas
        if label_classes_list is not None and legend_handles:
            legend_labels = [f"{label_classes_list[label_index]}: {(pixel_counts.get(label_classes_list[label_index], 0) * 0.01):.2f} km²" for label_index in added_labels]
            axes.legend(handles=legend_handles, labels=legend_labels, loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=3)

    return axes




In [86]:
import numpy as np
from scipy.ndimage import label
from scipy.ndimage import sum as ndi_sum

def remove_small_objects(mask, min_size):
    """
    Remove connected components that are smaller than min_size from the binary mask.
    
    Parameters:
    mask (numpy.ndarray): binary mask where objects are represented by ones and zeros.
    min_size (int): minimum size (number of pixels) that a connected component must have to be kept.
    
    Returns:
    numpy.ndarray: a new binary mask with the small connected components removed.
    """
    
    # Label each connected component in the mask
    labeled_mask, num_labels = label(mask)
    
    # For each labeled object, calculate its size
    sizes = ndi_sum(mask, labeled_mask, range(1, num_labels+1))
    
    # Create a new mask where small objects are removed
    new_mask = np.zeros_like(mask)
    removed_mask = np.zeros_like(mask)
    removed = 0
    kept = 0
    for label_num, size in enumerate(sizes):
        if size >= min_size:
            new_mask[labeled_mask == label_num+1] = 1
            kept += 1
        else:
            removed += 1
            removed_mask[labeled_mask == label_num+1] = 1
            
    return new_mask,removed_mask, kept, removed

In [88]:
def create_binary_masks(label_array, num_labels, image_shape):
    """
    Create binary masks for each label in the label array.

    :param label_array: 1D array of labels predicted by the model.
    :param num_labels: Number of labels.
    :param image_shape: Tuple representing the shape (height, width) of the image.
    :return: List of binary masks for each label.
    """
    label_array_reshaped = label_array.reshape(image_shape)
    masks = []
    for label in range(num_labels):
        mask = label_array_reshaped == label
        masks.append(mask)
    return masks

def get_polygons_from_model_output(model_output, num_labels, polygon_bound, image_shape, not_classified_label = None):
    """
    Processes the output from the model and generates polygons for each label.

    :param model_output: The output from the model, an array of labels for each pixel.
    :param num_labels: The number of distinct labels.
    :param polygon_bound: The bounding polygon in geographical coordinates.
    :param zone: Zone of the image.
    :param region: Region of the image.
    :param date: Date of the image.
    :return: A list of polygons with metadata.
    """
    all_polygons = []
    masks = create_binary_masks(model_output, num_labels, image_shape)
    for i, mask in enumerate(masks):
        label = i  # Assuming labels are integers starting from 0
        mask, removed_mask, objects_kept, objects_removed = remove_small_objects(mask, 10)
        print(f"Label {label}: {objects_kept} objects kept, {objects_removed} objects removed")
        _, polygons = get_polygons_from_mask(mask, label, polygon_bound, None, None, None, None)
        _, polygons_removed = get_polygons_from_mask(removed_mask, not_classified_label, polygon_bound, None, None, None, None)
        all_polygons.extend(polygons)
        all_polygons.extend(polygons_removed)

    
    return all_polygons

In [14]:
import matplotlib.pyplot as plt
from shapely.geometry import Polygon

def verify_polygon_formation(coords):
    # Crear un polígono a partir de las coordenadas
    poly = Polygon(coords)

    # Verificar si el polígono es válido y está cerrado
    if not poly.is_valid:
        print("El polígono no es válido.")
    if not poly.is_closed:
        print("El polígono no está cerrado.")

    # Dibujar el polígono
    fig, ax = plt.subplots()
    x, y = poly.exterior.xy  # Obtener las coordenadas x, y del polígono
    ax.plot(x, y, color="blue", alpha=0.7, linewidth=3, solid_capstyle='round', zorder=2)
    ax.set_title("Visualización del Polígono")
    plt.show()


In [73]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

def predict_with_confidence(model, data, threshold, label_classes, no_classified_index=1000):
    """
    Function to predict with a confidence threshold for machine learning models.

    Parameters:
    model: The trained machine learning model.
    data: The input data for prediction.
    threshold: The confidence threshold to filter predictions.
    label_classes: Array of label classes.
    no_classified_label: Label for predictions below the confidence threshold.

    Returns:
    confident_predictions: Predictions with confidence above the threshold.
    """

    # Obtener probabilidades de las clases
    probabilities = model.predict_proba(data)
    # Calcular la confianza como la probabilidad máxima
    confidences = np.max(probabilities, axis=1)
    # Obtener las clases predichas
    predictions = model.predict(data)

    # Crear un array para almacenar las predicciones finales
    confident_predictions = np.copy(predictions)

    # Asignar 'No Clasificado' a las predicciones con baja confianza
    for i in range(len(predictions)):
        if confidences[i] < threshold:
            confident_predictions[i] = no_classified_index

    return confident_predictions

def remap_indices(data, original_indices):
    """
    Remapea los índices en los datos para que sean consecutivos.

    Parameters:
    data: Array de datos con índices originales.
    original_indices: Lista de índices originales.

    Returns:
    remapped_data: Datos con índices remapeados.
    """
    remap_dict = {original_index: new_index for new_index, original_index in enumerate(sorted(original_indices))}
    remapped_data = np.array([remap_dict[i] for i in data])

    return remapped_data


# def predict_with_confidence(model, data, threshold, label_classes, label_indices, no_classified_label='No Clasificado'):
#     """
#     Function to predict with a confidence threshold for machine learning models.

#     Parameters:
#     model: The trained machine learning model.
#     data: The input data for prediction.
#     threshold: The confidence threshold to filter predictions.
#     label_classes: Array of label classes.
#     label_indices: Array of label indices corresponding to label_classes.
#     no_classified_label: Label for predictions below the confidence threshold.

#     Returns:
#     confident_predictions: Predictions with confidence above the threshold.
#     """
#     # Crear un diccionario para mapear índices a etiquetas
#     index_to_label_map = {index: label for label, index in zip(label_classes, label_indices)}

#     print("Índice a etiqueta:", index_to_label_map)

#     # Añadir 'No Clasificado' al mapeo si es necesario
#     no_classified_index = max(label_indices) + 1
#     index_to_label_map[no_classified_index] = no_classified_label

#     # Obtener probabilidades de las clases
#     probabilities = model.predict_proba(data)
#     # Calcular la confianza como la probabilidad máxima
#     confidences = np.max(probabilities, axis=1)
#     # Obtener las clases predichas
#     predictions = model.predict(data)

#     # Crear un array para almacenar las predicciones finales
#     confident_predictions = np.copy(predictions)

#     # Asignar 'No Clasificado' a las predicciones con baja confianza
#     for i in range(len(predictions)):
#         if confidences[i] < threshold:
#             confident_predictions[i] = no_classified_index
#         else:
#             # Asegurar que la predicción está en el diccionario
#             prediction_index = predictions[i]
#             if prediction_index in index_to_label_map:
#                 confident_predictions[i] = prediction_index
#             else:
#                 # Manejar el caso en el que la predicción no esté en el diccionario
#                 confident_predictions[i] = no_classified_index

#     return confident_predictions




In [ ]:
import ipywidgets as widgets
from ipywidgets import Output
from datetime import datetime
from dateutil.relativedelta import relativedelta
import json
from ipywidgets import Layout
from joblib import load
import sys

# Widget for image output
output_widget = Output()

# Widget para seleccionar la fecha de inicio
start_date_picker = widgets.DatePicker(
    description='Fecha de inicio',
    value= datetime.today().replace(month=1, day=1),
    disabled=False
)

# Widget para seleccionar la fecha de fin
end_date_picker = widgets.DatePicker(
    description='Fecha de fin',
    value=datetime.today().replace(day=1),
    disabled=False
)

# Función para actualizar el Dropdown de zonas
def update_zone_dropdown():
    zone_names = zone_data.get_polygon_list()
    zone_dropdown.options = zone_names

# Widget Dropdown para las zonas
zone_dropdown = widgets.Dropdown(
    description="Zona:",
    options=[],
    disabled=False
)
axes = None

update_zone_dropdown()  # Llamar a la función una vez para iniciar el ComboBox

def initialize_model_dropdown():
    # Update the model combobox
    models_path = './data/output/models/'
    model_names = [f for f in os.listdir(models_path) if f.endswith('.joblib')]
    model_dropdown.options = model_names
   
    if model_names:
        model_dropdown.value = model_names[0]
        model_dropdown.disabled = False
    else:
        model_dropdown.value = None
        model_dropdown.disabled = True

    show_model_json(model_dropdown.value)
    model_dropdown.observe(on_model_change, names='value')
    
def show_model_json(model_name):
    json_path = f"{models_path}{model_name.replace('_model.joblib', '_metadata.json')}"
    try:
        with open(json_path, 'r') as file:
            json_data = json.load(file)
            json_text_area.value = json.dumps(json_data, indent=4)
    except FileNotFoundError:
        json_text_area.value = "No se encontró el archivo JSON para el modelo seleccionado."
    
def on_model_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        model_name = change['new']
        show_model_json(model_name)

# Widget ComboBox for the models
model_dropdown = widgets.Dropdown(
    description="Modelo:",
    options=[],
    disabled=False,
    layout= Layout(width='100%', visibility='hidden')
)

# Text area for the JSON model
json_text_area = widgets.Textarea(
    description="JSON del Modelo:",
    disabled=True,
    layout= Layout(width='100%', height='200px', visibility='hidden')
)

apply_model_button = widgets.Button(description="Aplicar Modelo")
apply_model_button.layout.visibility = 'hidden'

results_text_area = widgets.Textarea(
    description="Resultados:",
    disabled=True,
    layout= Layout(width='100%', height='200px', visibility='hidden')
)

def convert_numpy_datetime_to_datetime(np_datetime):
    return datetime.utcfromtimestamp(np_datetime.astype('O')/1e9)

image_path_start = None
image_path_end = None
data_start = None
data_end = None
polygon = None

def on_process_button_clicked(b): 
    global image_path_start, image_path_end, data_start, data_end, axes, polygon
    process_button.disabled = True
    start_date = start_date_picker.value.strftime("%Y-%m-%d")
    end_date = end_date_picker.value.strftime("%Y-%m-%d")
    polygon = zone_data.get_polygon_by_name(zone_dropdown.value)
    data_start, image_path_start  = fetch_or_cache_stac_data_by_date(catalog, 
                                collections=["sentinel-2-l2a"],
                                bands=bands,
                                date=start_date,
                                bbox=polygon.bounds,
                                resolution=10,
                                stac_config=stac_cfg,
                                #crs="EPSG:4326",
                                crs = "EPSG:3857",
                                cache_dir=input_path,
                                log_level= LogLevel.Warning,
                                save_rgb_image=True)
    
  
    data_end, image_path_end  = fetch_or_cache_stac_data_by_date(catalog,
                                collections=["sentinel-2-l2a"],
                                bands=bands,
                                date=end_date,
                                bbox=polygon.bounds,
                                resolution=10,
                                stac_config=stac_cfg,
                                #crs="EPSG:4326",
                                crs = "EPSG:3857",
                                cache_dir=input_path,
                                log_level= LogLevel.Warning,
                                save_rgb_image=True)  
    
    if data_start is not None:
        start_date_picker.value = convert_numpy_datetime_to_datetime(data_start.time.values)
    if data_end is not None:
        end_date_picker.value = convert_numpy_datetime_to_datetime(data_end.time.values)

    process_button.disabled = False
    if image_path_start is not None and image_path_end is not None:
        axes = show_rgb_images(output_widget, image_path_start, image_path_end,)
        model_dropdown.layout.visibility = 'visible'
        json_text_area.layout.visibility = 'visible'
        apply_model_button.layout.visibility = 'visible'
        initialize_model_dropdown()  
        
def apply_selected_model(model_name, image_path_start, image_path_end):
    global polygon
    confidence_threshold = None
    confidence_threshold_multiclass = 0.6
    confidence_threshold_binary = 0.8

    model_path = f'./data/output/models/{model_name}'
    model_data = load(model_path)
    model = model_data['model']
    min_vals = model_data['min_vals']
    max_vals = model_data['max_vals'] 
    label_classes = model_data['label_classes']

    json_path = model_path.replace('_model.joblib', '_metadata.json')
    with open(json_path, 'r') as file:
        model_metadata = json.load(file) 
        model_bands = model_metadata['feature_names']
        label_indexs = model_metadata['class_labels']
    
    if len(label_classes) == 2:
        confidence_threshold = confidence_threshold_binary
    else:
        confidence_threshold = confidence_threshold_multiclass

    no_classified_index = None
    print(f"Modelo cargado: {model_path}")
    if 'No Clasificado' not in label_classes:
        label_classes = np.append(label_classes, 'No Clasificado')
        #Agregamos indice para No Clasificado igual al mayor indice + 1
        label_indexs = np.append(label_indexs, max(label_indexs) + 1)
        no_classified_index = max(label_indexs)
        print(f"Índices de etiquetas remapeados: {remap_indices(label_indexs, label_indexs)}")
        print(f" (No Clasificado = {no_classified_index})")
    else:
        no_classified_index = label_indexs[label_classes.tolist().index('No Clasificado')]

    preprocess_data_start = preprocess_image_data(data_start, model_bands, min_vals, max_vals)
    preprocess_data_end = preprocess_image_data(data_end, model_bands, min_vals, max_vals)

    print(f"Umbral de confianza: {confidence_threshold}")
    model_output_start = predict_with_confidence(model, preprocess_data_start, confidence_threshold, label_classes, no_classified_index)
    model_output_end = predict_with_confidence(model, preprocess_data_end, confidence_threshold, label_classes, no_classified_index)

    print(f"Índices de etiquetas originales: {label_indexs}")
    print(f"Clases de etiquetas: {label_classes}")
    print(f"Índices de etiquetas remapeados: {remap_indices(label_indexs, label_indexs)}")
    print(model_output_start)
    model_output_start = remap_indices(model_output_start, label_indexs)
    print(model_output_start)
    model_output_end = remap_indices(model_output_end, label_indexs)


    results_text_area.layout.visibility = 'visible'
    results_text_area.value = f"Modelo: {model_name}\n\n"
    results_text_area.value += f"Fecha inicial: {start_date_picker.value}\n"
    results_text_area.value += f"Fecha final: {end_date_picker.value}\n\n"
    results_text_area.value += f"Resultados:\n"
    results_text_area.value += f"Fecha inicial: {model_output_start}\n"
    results_text_area.value += f"Fecha final: {model_output_end}\n"
    count_start = count_pixels_by_label(model_output_start, label_classes)
    count_end = count_pixels_by_label(model_output_end, label_classes)

    # Contar cambios de etiquetas entre las dos imágenes
    label_changes = count_label_changes(model_output_start, model_output_end, label_classes)

    # Agregar estos resultados a tu área de texto
    results_text_area.value += f"Conteo de píxeles por etiqueta (inicio): {count_start}\n"
    results_text_area.value += f"Conteo de píxeles por etiqueta (fin): {count_end}\n"
    results_text_area.value += f"Cambios de etiquetas: {label_changes}\n"
    
    image_shape = (data_start.dims['y'], data_start.dims['x'])
    polygons_start = get_polygons_from_model_output(model_output_start, len(label_classes), polygon, image_shape, no_classified_index)
    polygons_end = get_polygons_from_model_output(model_output_end, len(label_classes), polygon, image_shape, no_classified_index)
    show_rgb_images(output_widget, image_path_start, image_path_end, label_classes, polygons_start, polygons_end, count_start, count_end)
    


def on_apply_model_button_clicked(b):
    model_name = model_dropdown.value
    apply_model_button.disabled = True
    apply_selected_model(model_name, image_path_start, image_path_end)
    print("Modelo aplicado")
    apply_model_button.disabled = False
    
apply_model_button.on_click(on_apply_model_button_clicked)

process_button = widgets.Button(description="Process")
process_button.on_click(on_process_button_clicked)

display(widgets.HBox([start_date_picker, end_date_picker, zone_dropdown, process_button]))
display(output_widget)
display( apply_model_button, model_dropdown, json_text_area, results_text_area)

Output()

Button(description='Aplicar Modelo', layout=Layout(visibility='hidden'), style=ButtonStyle())

Dropdown(description='Modelo:', layout=Layout(visibility='hidden', width='100%'), options=(), value=None)

Textarea(value='', description='JSON del Modelo:', disabled=True, layout=Layout(height='200px', visibility='hi…

Textarea(value='', description='Resultados:', disabled=True, layout=Layout(height='200px', visibility='hidden'…

Modelo cargado: ./data/output/models/random_forest_no_clouds_best_model.joblib
Índices de etiquetas remapeados: [0 1 2 3]
 (No Clasificado = 4)
['rededge2', 'rededge3', 'blue', 'nir08', 'nir', 'swir16', 'swir22', 'rededge1', 'red', 'green', 'nir09']
['rededge2', 'rededge3', 'blue', 'nir08', 'nir', 'swir16', 'swir22', 'rededge1', 'red', 'green', 'nir09']
Umbral de confianza: 0.6
Índices de etiquetas originales: [0 2 3 4]
Clases de etiquetas: ['Bosque' 'Quemado' 'Talado' 'No Clasificado']
Índices de etiquetas remapeados: [0 1 2 3]
[3 3 3 ... 0 0 0]
[2 2 2 ... 0 0 0]
Label 0: 48 objects kept, 144 objects removed
Label 1: 40 objects kept, 139 objects removed
Label 2: 198 objects kept, 462 objects removed
Label 3: 283 objects kept, 2722 objects removed
